In [ ]:
#ChromeOS
sudo apt install -y autoconf libaio-dev libbluetooth-dev libbrlapi-dev libbz2-dev libcap-dev libcap-ng-dev libcurl4-gnutls-dev libepoxy-dev libfdt-dev libgbm-dev libgles2-mesa-dev libglib2.0-dev libgtk-3-dev libibverbs-dev libjpeg8-dev liblzo2-dev libncurses5-dev libnuma-dev librbd-dev librdmacm-dev libsasl2-dev libsdl1.2-dev libsdl2-dev libseccomp-dev libsnappy-dev libssh2-1-dev libspice-server-dev libspice-server1 libtool libusb-1.0-0 libusb-1.0-0-dev libvde-dev libvdeplug-dev libvte-dev libxen-dev valgrind xfslibs-dev xutils-dev zlib1g-dev libusbredirhost-dev usbredirserver

#virglrenderer
git clone git://git.freedesktop.org/git/virglrenderer
cd virglrenderer
./autogen.sh
make -j7
sudo make install

#https://unix.stackexchange.com/questions/268460/how-to-mount-qcow2-image
#Step 1 - Enable NBD on the host
sudo modprobe nbd max_part=8
#Step 2 - Connect the QCOW2 as a network block device
sudo qemu-nbd --connect=/dev/nbd0 /var/lib/libvirt/images/ChromeOS.qcow2
#Step 3 - List partitions inside the QCOW2
sudo fdisk /dev/nbd0 -l
#Step 4 - Mount the partition from the VM
#sudo mount /dev/nbd0p1 /mnt/somepoint/


#COS
sudo apt-get update
sudo apt-get install -y figlet 
sudo apt-get install -y pv
sudo apt-get install -y cgpt
cd /media/user/Ventoy/ChromeOS
sudo bash chromeos-install.sh -src rammus_recovery.bin -dst /dev/nbd0

#Step 5 - After you're done, unmount and disconnect
#sudo umount /mnt/somepoint/
sudo qemu-nbd --disconnect /dev/nbd0
sudo rmmod nbd


cd /media/user/Ventoy/ChromeOS
/usr/bin/qemu-system-x86_64 \
    -enable-kvm \
    -m 2G \
    -smp 4 \
    -hda /media/user/Ventoy/ChromeOS/rammus_recovery.bin \
    -vga virtio \
    -net nic,model=virtio \
    -net user,hostfwd=tcp:127.0.0.1:9222-:22 \
    -usb -usbdevice keyboard \
    -usbdevice mouse \
    -device virtio-gpu-pci,virgl \
    -display gtk,gl=on